# Подготовка данных

Предполагаем что все данные лежат в папке data.
Исходные данные представляют из себя лемматизированные тексты патентов в папке data/raw. Сколько патентов, столько и файлов. Ниже приводится процедура по конвертированию этих файлов в один большой. Работа с одним файлом удобна для разных алгоритмов и влияет на скорость чтения данных с диска. Слова в текстах отделены пробелами, а предложения точками. Причем эти тексты результат обработки лемматизатором, который кроме прочего отделяет знаки препинания от слов и отделяет их пробелами.

In [ ]:
from glob import glob

# получаем имена файлов в дирректории
fnames = glob('./raw/*')

process - функция, которая обрабатывает часть файлов и сохраняет результат обработки ввиде одного файла с именем data/corpus_json.txt. Функция вначале разбивает текст на предложения, затем для каждого слова в предложении проверяет что оно не является стоп словом и что длина слова больше 1. Обработанный таким образом документ представляет список предложений, каждое из которых является саиском слов. Затем обработанный таким образом документ сохраняется ввиде стори json: (document_id, document) и сохраняется в файл data/i.txt ввиде строки json и так для каждого документа.

Стоп слова хранятся в файле data/StopWords.txt

In [ ]:
import io, json
from os.path import basename
from nltk.tokenize import sent_tokenize

with io.open(join(cur_dir, 'StopWords.txt'), 'r', encoding='utf8') as f:
    stop_words = f.read().splitlines()

def process(fnames):
    with io.open('../data/corpus.txt', 'w', encoding='utf8') as fw:
        for fn in fnames:
            with io.open(fn, encoding='utf8') as fr:
                text = fr.read()
            sents = sent_tokenize(text)
            sents = [[w for w in s.split() if w not in stop_list and len(w)>1] 
                     for s in sents]
            sents = [s for s in sents if len(s)]
            s = json.dumps((basename(fn).split('.')[0], sents))            
            fw.write(s + u'\n')

Затем файл corpus_json.txt сжимается с помощью gzip для уменьшения занимаемого места на диске, сжатие файла не влияет на скорость его дальнейшего чтения.

Далее приводятся процедуры для чтения файла corpus_json.txt.gz с помощью gensim

In [ ]:
import ujson
import gensim
from gzip import GzipFile

def iter_docs():
    with GzipFile('../data/corpus_json.txt.gz', 'r') as fr:
        for line in fr:
            _id, sents = ujson.loads(line)
            yield _id, sents
    
def iter_sents():
    for _id, sents in iter_docs():
        for s in sents:
            yield s
                
class Sentences(object):
    def __iter__(self):
        for sent in iter_sents():
            yield sent

Получаем биграммы

In [ ]:
bigram = gensim.models.Phrases(iter_sents())

Сохраняем биграммы на диск для дальнейшего использования

In [ ]:
bigram.save('../data/bigram')

Посмотреть полученные биграммы для первых 100 предложений можно так: 

In [ ]:
for phrase, score in bigram.export_phrases(islice(iter_sents(), 100)):
    print(phrase)

Далее мы преобразуем биграммы в формат наиболее быстрый для их применения к тексту и сохраняем.

In [ ]:
bigram_ph = gensim.models.phrases.Phraser(bigram)
bigram_ph.save('../data/bigram_ph')

Получаем триграммы как результат применения процедуры получения биграмм к ранее полученному тексту с биграммами. Когда биграмма объединяется с одним словом получается триграмма. А когда объединяются две биграммы, получается 4-х грамма

In [ ]:
trigram = gensim.models.Phrases(bigram_ph[iter_sents()])
trigram.save('../data/trigram')

Таже процедура применяется к триграммам

In [ ]:
trigram_ph = gensim.models.phrases.Phraser(trigram)
trigram_ph.save('../data/trigram_ph')

# Отбор триграмм 

На данный момент оперативная память компьютера может быть уже заполнена, поэтому есть смысл завершить процесс и открыть новый, загрузив в него trigram_ph, который не занимает много места:

In [ ]:
trigram_ph = gensim.models.phrases.Phrases.load('../data/trigram_ph')

В trigram_ph.phrasegrams.items() хранятся пары ((слово1, слово2), (tf, score)), где tf - term friquency, т.е. частота встречи слова во всем корпусе, score - это метрика для оценки того насколько часто слова входящие в n-грамму встречаются совместно нежели по отдельности. Эта метрика считается по формуле

(count(worda followed by wordb) - min_count) * N / (count(worda) * count(wordb)) > threshold, where N is the total vocabulary size

В цикле каждая n-грамма отбирается по погрогам min_count, threshold. Значения порогов выбираются на усмотрение по качеству получаемых результатов. Также проверяется что в каждом составляющем слове есть хотябы одна буква.

In [ ]:
import pandas as pd

min_count, min_score = 5, 10

_list = []
for k,v in trigram_ph.phrasegrams.items():
    if v[0] > min_count and v[1] > threshold:
        the_string = '_'.join(k)
        if re.search('[а-яА-Яa-zA-Z]', k[0]) and re.search('[а-яА-Яa-zA-Z]', k[1]):
            _list.append([the_string, v[0], v[1]])
        
df = pd.DataFrame(_list)

Проверить количество полученных триграмм можно так:

In [ ]:
len(df[0].unique())

Сохраняем коллокации в архив, отсортированные по имени для удобства просмотра

In [ ]:
with GzipFile('../data/collocations_trigrams.txt.gz', 'w') as f:
    for tag in sorted(df[0].unique()):
        f.write(tag + '\n')

# Получение классов коллокаций

Когда min_count и threshold подобраны, можно начать получение уже итогового trigram_ph с фиксированными min_count и threshold